In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom

from pymedphys_analysis.tpscompare import load_and_normalise_mephysto
from pymedphys_dicom.dicom import depth_dose, profile

In [ ]:
ROOT_DIR = Path(r"S:\Physics\Monaco\Model vs Measurement Comparisons")

MONACO_DICOM_DIR = ROOT_DIR.joinpath(r"Beam Models\CCA Monaco Collapsed Cone")
DOSECHECK_DICOM_DIR = ROOT_DIR.joinpath(r"Beam Models\DoseCHECK")
MEASUREMENTS_DIR = ROOT_DIR.joinpath(r"Measurements\RCCC\Photons")
RESULTS = ROOT_DIR.joinpath(r"Results\RCCC\dosecheck\Collapsed Cone")

MONACO_DOSE_DIR = MONACO_DICOM_DIR.joinpath("DICOM dose exports")

In [ ]:
calibrated_doses_table = pd.read_csv(MEASUREMENTS_DIR.joinpath('AbsoluteDose.csv'), index_col=0)
calibrated_doses = calibrated_doses_table['d10 @ 90 SSD']
calibrated_doses

In [ ]:
wedge_transmission_table = pd.read_csv(MEASUREMENTS_DIR.joinpath('WedgeTransmissionFactors.csv'), index_col=0)
data_column_name = wedge_transmission_table.columns[0]
wedge_transmissions = wedge_transmission_table[data_column_name]
wedge_transmissions

In [ ]:
output_factors = pd.read_csv(MEASUREMENTS_DIR.joinpath('OutputFactors.csv'), index_col=0)
output_factors

In [ ]:
keys = [
    path.stem
    for path in MONACO_DOSE_DIR.glob('*.dcm')
]

keys

In [ ]:
regex_string = r'(\d\dMV(\bFFF\b)?) (\d\dx\d\d) ((\bOpen\b)|(\bWedge\b))'

def get_energy_field_block(key):
    match = re.match(regex_string, key)
    return match.group(1), match.group(3), match.group(4)

In [ ]:
absolute_doses = {}

for key in keys:
    energy, field, block = get_energy_field_block(key)
        
    if block == 'Wedge':
        wtf = wedge_transmissions[energy]
    else:
        wtf = 1
        
    output_factor = output_factors[f'{field} {block}'][energy]
    calibrated_dose = calibrated_doses[energy]
    
    absolute_dose = calibrated_dose * output_factor * wtf
    absolute_doses[key] = absolute_dose
    

absolute_doses

In [ ]:
getter = operator.itemgetter('displacement', 'dose')

In [ ]:
absolute_scans_per_field = load_and_normalise_mephysto(
    MEASUREMENTS_DIR, r'(\d\dMV(\bFFF\b)? \d\dx\d\d ((\bOpen\b)|(\bWedge\b)))\.mcc', absolute_doses, 100)

In [ ]:
new_keys = list(absolute_scans_per_field.keys())
new_keys

In [ ]:
assert new_keys == keys

In [ ]:
def load_dicom_files(directory, keys):
    dicom_file_map = {
        key: directory.joinpath(f'{key}.dcm')
        for key in keys
    }
    
    dicom_dataset_map = {
        key: pydicom.read_file(str(dicom_file_map[key]), force=True)
        for key in keys
    }
    
    return dicom_dataset_map

In [ ]:
monaco_dicom_dataset_map = load_dicom_files(MONACO_DOSE_DIR, keys)
dosecheck_dicom_dataset_map = load_dicom_files(DOSECHECK_DICOM_DIR, keys)

In [ ]:
dicom_plan = pydicom.read_file(str(MONACO_DICOM_DIR.joinpath('plan.dcm')), force=True)

In [ ]:
def plot_one_axis(ax, displacement, meas_dose, model_dose):
    diff = 100 * (model_dose - meas_dose) / meas_dose
    
    lines = []
    
    lines += ax.plot(displacement, meas_dose, label='Measured Dose')
    lines += ax.plot(displacement, model_dose, label='Model Dose')
    ax.set_ylabel('Dose (Gy / 100 MU)')
    
    x_bounds = [np.min(displacement), np.max(displacement)]
    ax.set_xlim(x_bounds)

    ax_twin = ax.twinx()

    lines += ax_twin.plot(displacement, diff, color='C3', alpha=0.5, label=r'% Residuals [100 $\times$ (Model - Meas) / Meas]')
    ax_twin.plot(x_bounds, [0, 0], '--', color='C3', lw=0.5)
    ax_twin.set_ylabel(r'% Dose difference [100 $\times$ (Model - Meas) / Meas]')

    labels = [l.get_label() for l in lines]
    
    ax.legend(lines, labels, loc='lower left')
    
    return ax_twin



def plot_tps_meas_diff(displacement, meas_dose, internal_tps_dose, external_tps_dose):
    fig, ax = plt.subplots(1, 2, figsize=(16,6), sharey=True)
    ax[1].yaxis.set_tick_params(which='both', labelbottom=True)

    ax_twin = list()
    
    ax_twin.append(plot_one_axis(ax[0], displacement, meas_dose, internal_tps_dose))
    ax_twin.append(plot_one_axis(ax[1], displacement, meas_dose, external_tps_dose))
    
    ax_twin[1].get_shared_y_axes().join(ax_twin[1], ax_twin[0])
    ax_twin[1].set_ylim([-5, 5])
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.4, top=0.86)
    
    return fig, ax


def plot_pdd_diff(key, dicom_plan):
    depth, meas_dose = getter(absolute_scans_per_field[key]['depth_dose'])
    internal_tps_dose = depth_dose(depth, monaco_dicom_dataset_map[key], dicom_plan) / 10
    external_tps_dose = depth_dose(depth, dosecheck_dicom_dataset_map[key], dicom_plan) / 10

    fig, ax = plot_tps_meas_diff(depth, meas_dose, internal_tps_dose, external_tps_dose)
    fig.suptitle(f'Depth Dose Comparisons | {key}', fontsize="x-large")
    ax[0].set_title("Monaco Collapsed Cone")
    ax[1].set_title("DoseCHECK")
    

for key in keys:
    plot_pdd_diff(key, dicom_plan)
    filename = RESULTS.joinpath(f'{key}_pdd.png')
    plt.savefig(filename)
    plt.show()

In [ ]:
def plot_profile_diff(key, depth, direction):
    displacement, meas_dose = getter(absolute_scans_per_field[key]['profiles'][depth][direction])
    
    internal_tps_dose = profile(displacement, depth, direction, monaco_dicom_dataset_map[key], dicom_plan) / 10
    external_tps_dose = profile(displacement, depth, direction, dosecheck_dicom_dataset_map[key], dicom_plan) / 10

    fig, ax = plot_tps_meas_diff(displacement, meas_dose, internal_tps_dose, external_tps_dose)
    fig.suptitle(f'{direction.capitalize()} Profile Comparisons | {key} | Depth: {depth} mm', fontsize="x-large")
    ax[0].set_title("Monaco Collapsed Cone")
    ax[1].set_title("DoseCHECK")
    

for key in keys:
    depths = absolute_scans_per_field[key]['profiles'].keys()
    for depth in depths:
        for direction in ['inplane', 'crossplane']:
            plot_profile_diff(key, depth, direction)
            filename = RESULTS.joinpath(f'{key}_profile_{depth}mm_{direction}.png')
            plt.savefig(filename)
            plt.show()